In [1]:
import pandas as pd
file_name = 'bank.csv'

def prepare_str(s):
    return s[1:-1].lower().strip()

file = pd.read_csv(file_name, encoding='utf-8', sep=";", quoting=3,  
                   converters={'"job"':prepare_str,
                              '"marital"':prepare_str,
                              '"education"':prepare_str,
                              '"default"':prepare_str,
                              '"housing"':prepare_str,
                              '"loan"':prepare_str,
                              '"contact"':prepare_str,
                              '"month"':prepare_str,
                              '"day_of_week"':prepare_str,
                              '"poutcome"':prepare_str,
                              '"y"':prepare_str,})

file.columns = [ prepare_str(s) for s in file.columns]

In [2]:
# fixing data entry typos 

marital_c = file['marital']
marital_c.mask(marital_c == "divorceded", other="divorced", inplace=True)

default_c = file['default']
default_c.mask(default_c == "noo", other="no", inplace=True)

housing_c = file['housing']
housing_c.mask(housing_c == "yess", other="yes", inplace=True)

# setting to NaN, then later fill with column-wise mean value
duration_c = file['duration']
# getting error with mean function on duration, going to use mode instead
duration_mode = file["duration"].value_counts().axes[0][0]
duration_c.mask(duration_c == "-", other=duration_mode, inplace=True)

# sanity tests(dropping impossible values)

# assuming there isn't anyone whose age > 120
idx = file[(file['age'] > 120)].index.values
file.drop(idx, inplace=True)

# if duration = 0 then y must be 'no'
idx = file[(file['duration'] == 0) &
               (file['y'] == 'yes')].index.values
file.drop(idx, inplace=True)

# if pdays == 999 then poutcome should not be 'success'
idx = file[(file['pdays'] == 999) &
               (file['poutcome'] == 'success')].index.values
file.drop(idx, inplace=True)

# filling NA values with column-wise mean

file.fillna(file.mean(axis=0), inplace=True)

In [3]:
file.to_csv("cleaned_bank.csv", sep=";", index=False)

In [4]:
file.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30.0,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39.0,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25.0,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
3,38.0,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
4,47.0,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no
